In [1]:
!pip install keras-core
!git clone https://github.com/anas-rz/focalnet-keras-core

fatal: destination path 'focalnet-keras-core' already exists and is not an empty directory.


In [1]:
import os
os.environ['KERAS_BACKEND'] = 'torch' #jax, tensorflow

import sys
sys.path.append('./focalnet-keras-core/')

In [2]:
import keras_core as keras

import tensorflow_datasets as tfds
import tensorflow as tf

Using PyTorch backend.


In [3]:
from focalnet_keras_core import focalnet_tiny_lrf

In [4]:
train_ds, validation_ds = tfds.load(
    "tf_flowers",
    split=["train[:85%]", "train[85%:]"],
    as_supervised=True,
)

In [5]:
RESIZE_TO = 384
CROP_TO = 224
BATCH_SIZE = 64
STEPS_PER_EPOCH = 10
AUTO = tf.data.AUTOTUNE  # optimise the pipeline performance
NUM_CLASSES = 5  # number of classes
SCHEDULE_LENGTH = (
    500  # we will train on lower resolution images and will still attain good results
)
SCHEDULE_BOUNDARIES = [
    200,
    300,
    400,
]  # more the dataset size the schedule length increase
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 256
num_epochs = 100
num_classes = 5  # number of classes


In [6]:
@tf.function
def preprocess_train(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.resize(image, (RESIZE_TO, RESIZE_TO))
    image = tf.image.random_crop(image, (CROP_TO, CROP_TO, 3))
    image = image / 255.0
    return (image, label)


@tf.function
def preprocess_test(image, label):
    image = tf.image.resize(image, (CROP_TO, CROP_TO))
    image = image / 255.0
    return (image, label)


In [7]:
# Training pipeline
pipeline_train = (
    train_ds.shuffle(10000)
    # .repeat(repeat_count)  # Repeat dataset_size / num_steps
    .map(preprocess_train, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

# Validation pipeline
pipeline_validation = (
    validation_ds.map(preprocess_test, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

In [8]:
model = focalnet_tiny_lrf(num_classes = num_classes)

In [9]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │       0 │ -                    │
│ (InputLayer)        │ 3)                │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ patch_embed.proj    │ (None, 56, 56,    │   4,704 │ input_layer[0][0]    │
│ (Conv2D)            │ 96)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ reshape (Reshape)   │ (None, 3136, 96)  │       0 │ patch_embed.proj[0]… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ patch_embed.norm    │ (None, 3136, 96)  │     192 │ reshape[0][0]        │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dropout (Dropout)   │ (None, 3136, 96)  │       0 │ patch_embed.norm[0]… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layers.0.blocks.0.… │ (None, 3136, 96)  │     192 │ dropout[0][0]        │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ reshape_1 (Reshape) │ (None, 56, 56,    │       0 │ layers.0.blocks.0.n… │
│                     │ 96)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layers.0.blocks.0.… │ (None, 56, 56,    │  45,604 │ reshape_1[0][0]      │
│ (FocalModulation)   │ 96)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ reshape_2 (Reshape) │ (None, 3136, 96)  │       0 │ layers.0.blocks.0.m… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ stochastic_depth    │ (None, 3136, 96)  │       0 │ reshape_2[0][0]      │
│ (StochasticDepth)   │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add (Add)           │ (None, 3136, 96)  │       0 │ dropout[0][0],       │
│                     │                   │         │ stochastic_depth[0]… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ reshape_3 (Reshape) │ (None, 56, 56,    │       0 │ add[0][0]            │
│                     │ 96)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layers.0.blocks.0.… │ (None, 56, 56,    │     192 │ reshape_3[0][0]      │
│ (LayerNormalizatio… │ 96)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layers.0.blocks.0.… │ (None, 56, 56,    │  37,248 │ layers.0.blocks.0.n… │
│ (Dense)             │ 384)              │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dropout_2 (Dropout) │ (None, 56, 56,    │       0 │ layers.0.blocks.0.m… │
│                     │ 384)              │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layers.0.blocks.0.… │ (None, 56, 56,    │  36,960 │ dropout_2[0][0]      │
│ (Dense)             │ 96)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dropout_3 (Dropout) │ (None, 56, 56,    │       0 │ layers.0.blocks.0.m… │
│                     │ 96)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ stochastic_depth_1  │ (None, 56, 56,    │       0 │ dropout_3[0][0]    

 Total params: 27,830,933 (106.17 MB)

 Trainable params: 27,830,933 (106.17 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
optimizer = keras.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )
model.compile(
        optimizer=optimizer,
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
            # keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
        ],
    )
history = model.fit(
    pipeline_train,
    batch_size=BATCH_SIZE,
    epochs=5,
    # steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=pipeline_validation,
    # callbacks=train_callbacks,
)

Epoch 1/5
49/49 ━━━━━━━━━━━━━━━━━━━━ 146s 3s/step - accuracy: 0.2165 - loss: 6.6212 - val_accuracy: 0.2036 - val_loss: 1.6530
Epoch 2/5
 6/49 ━━━━━━━━━━━━━━━━━━━━ 1:42 2s/step - accuracy: 0.2389 - loss: 1.7738